In [64]:
import pandas as pd
import nltk
import string
import json

from nltk import tokenize
from matplotlib import pyplot as plt

In [2]:
lyrics_df = pd.read_csv("data/data_raw.csv", sep=";")

In [3]:
lyrics_df.head()

,Unnamed: 0,artista,musica,letra,selecionado
0,0,Roberto Carlos,Nossa Senhora,Cubra-me com seu manto de amor\nGuarda-me na p...,1
1,1,Roberto Carlos,Amor Sem Limite,Quando a gente ama alguém de verdade\nEsse amo...,1
2,2,Roberto Carlos,Como É Grande O Meu Amor Por Você,Eu tenho tanto pra lhe falar\nMas com palavras...,1
3,3,Roberto Carlos,Amigo,"Você meu amigo de fé, meu irmão camarada\nAmig...",1
4,4,Roberto Carlos,Emoções,Quando eu estou aqui\nEu vivo esse momento lin...,1


---

## Palavra

In [7]:
full_text = " ".join(lyrics_df["letra"].values)

In [19]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/tiago/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/tiago/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [15]:
word_tokenize = tokenize.word_tokenize(full_text.lower(), language='portuguese')    

In [29]:
word_tokenize = [w for w in word_tokenize if w not in list(string.punctuation)]

In [30]:
words = pd.Series(word_tokenize)

In [34]:
word_count = words.value_counts()

In [37]:
word_count.shape[0]

12654

In [47]:
word_count[word_count >= 30].shape[0]

820

In [52]:
word_count = word_count[word_count >= 30].reset_index()

In [55]:
word_count["word_len"] = word_count["index"].apply(len)

In [58]:
word_count = word_count[word_count["word_len"] != 1]

In [62]:
palavra_cards = [dict(palavra=w) for w in word_count["index"]]

In [66]:
palavra_json = json.dumps(palavra_cards)

In [71]:
palavra_json = 'var palavra = ' + palavra_json + ";"

---

## Complete

In [75]:
lyrics_paragraphs = lyrics_df["letra"].apply(lambda x: x.split("\n\n"))

In [91]:
lyrics_paragraphs[0][0]

'Cubra-me com seu manto de amor\nGuarda-me na paz desse olhar\nCura-me as feridas e a dor me faz suportar\nQue as pedras do meu caminho\nMeus pés suportem pisar\nMesmo ferido de espinhos me ajude a passar'

In [108]:
complete_list = []
for lyric in lyrics_paragraphs:
    complete_list += [
        dict(inicio="\n".join(p.split("\n")[:-1]), resposta=p.split("\n")[-1])
        for p in set(lyric)
        if len(p.split("\n")) > 1
    ]

In [111]:
complete_json = json.dumps(complete_list)
complete_json = "var complete = " + complete_json + ";"

---

## Adivinha

In [128]:
artist_list = lyrics_df.apply(lambda x: x["artista"] + " - " + x["musica"], axis=1).values

In [114]:
lyrics_paragraphs[0][0]

'Cubra-me com seu manto de amor\nGuarda-me na paz desse olhar\nCura-me as feridas e a dor me faz suportar\nQue as pedras do meu caminho\nMeus pés suportem pisar\nMesmo ferido de espinhos me ajude a passar'

In [129]:
advinha_list = []
for i, lyric in enumerate(lyrics_paragraphs):
    advinha_list += [
        dict(verse="\n".join(p.split("\n")[0:2]), artist=artist_list[i])
        for p in set(lyric)
        if len(p.split("\n")) > 1
    ]

In [131]:
advinha_json = json.dumps(advinha_list)
advinha_json = "var adivinha = " + advinha_json + ";"

---

## Mimica

In [139]:
mimica_list = lyrics_df.apply(lambda x: dict(title=x["musica"], artist=x["artista"]), axis=1).values.tolist()

In [140]:
mimica_json = json.dumps(mimica_list)
mimica_json = "var mimica = " + mimica_json + ";"

---

## Join all cards

In [142]:
json_data = "\n".join([palavra_json, complete_json, advinha_json, mimica_json])

In [144]:
with open("data/data.js", "w") as outfile:
    outfile.write(json_data)